In [1]:
# Import libraries
import os
import json

In [2]:
# Example with loading a JSON file
def load(path):
    # Open the JSON file
    with open(path, 'r') as file:
        # Load the JSON data
        jsonObj = json.load(file)
        return jsonObj
 
def save(obj, path):   
    json_data = json.dumps(obj)

    with open(path, 'w') as file:
        # Write JSON data to the file
        json.dump(obj, file)
        print('Saved file')

In [3]:
# Process BigNumber Course Assessment (.ba)
def process_assessment(path):
    assessmentObj = dict({
        "fileName": "",
        "file": "",
        "title": "",
        "duration": "",
        "quiz": "",
        "url": "",
        "assessmentDetails": dict({
            "organization": []
        })  
    })
        
    l = dict(
        {
            "qt": "questionTitle",
            "t": "type",
            "q": "question",
            "o": "options",
            "a": "answerKey",
            "h": "hint",
            "c": "choice",
            "b": "body"
        }
    )

    template_preprocess = -1
    # Open the file in read mode
    with open(path, 'r') as file:
        # Read each line and print it if it's not empty
        for line in file:
            line = line.strip()
            if line:
                
                # Assessment setup
                if line == '--- template_end ---':
                    template_preprocess = 0
                    
                if template_preprocess == 1:
                    # print(line)
                    key, value = line.split("::")
                    # pair = [key.strip(), value.strip()]
                    key = key.strip()
                    value = value.strip()
                    
                    # Casting is done is done at playlist level
                    if(value == "true"): value = True
                    if(value == "false"): value = False
                    
                    if type(assessmentObj[key]) == str:
                        assessmentObj[key] = value
                        
                if line == '--- template_start ---':
                    template_preprocess = 1
            
                
                # Assessment organization
                # Start processing questions, answers, hints after template_end..
                if template_preprocess == 0:
                    if line != "--- template_end ---":
                        detail = line.split('::')
                            
                        if len(detail) > 1:
                                                    
                            k = detail[0].strip()
                            v = detail[1].strip()

                            # Casting is done is done at playlist level                          
                            if(v == "true"):  v = True
                            if(v == "false"):  v = False
                            
                            if l[k] == "questionTitle":
                                question_type = ""
                                new_question = dict({
                                    "questionTitle": "",
                                    "details": dict({
                                        "options": [],
                                        "answerKey": "",
                                        "hint": ""
                                    })
                                })
                                append_next = 1
                            
                            # l[] is for the lookup
                            if l[k] == "questionTitle" :
                                new_question[l[k]] = v
                            
                            elif l[k] == "type":
                                question_type = v
                                
                                # Setup options for the question
                                if len(new_question['details']['options']) == 0:
                                    new_question['details']['options'] = []
                                
                                # If simple text input, append options straightaway
                                if question_type == "text":
                                    new_question['details']['options'].append(
                                        dict({
                                            "type": question_type,
                                            "text": ""
                                        }))
                            
                            elif l[k] == "options":
                                if question_type == "radio":
                                    new_question['details']['options'].append(
                                        dict({
                                            "type": question_type,
                                            "choices": []
                                        }))
                                                                    
                                
                            elif l[k] == "choice":
                                new_question['details']['options'][0]['choices'].append(
                                    {"text": v}
                                )
                            
                            else: 
                                new_question['details'][l[k]] = v
                            
                            if append_next == 1:
                                append_next = 0
                                assessmentObj['assessmentDetails']['organization'].append(new_question)
                      
    return assessmentObj, assessmentObj['fileName']


In [39]:
mod = 1
course_file_path = f'bignumber-material/SPR/mod{mod}'
url_prefix = f'http://localhost:3001'
url_prefix_math = f'http://localhost:3000'

# Process BigNumber Course Videoplaylist (.bpls)
def process_playlist_section(path):
    sectionObj = dict(
        {
            "fileName": "",
            "section": "",
            "children": []
        }
    )

    template_preprocess = -1
    i_file = 0
    with open(path, 'r') as file:
            for line in file:
                line = line.strip()
                    
                # Section setup
                if line == '--- template_end ---':
                    template_preprocess = 0
                    
                if template_preprocess == 1:
                    key, value = line.split("::")
                    # pair = [key.strip(), value.strip()]
                    key = key.strip()
                    value = value.strip()
                    
                    if(value == "true"): value = True
                    if(value == "false"): value = False
                    
                    if type(sectionObj[key]) == str:
                        sectionObj[key] = value    
                    
                if line == '--- template_start ---':
                    template_preprocess = 1
                    
                    
                # Playlist processing
                # Start processing file, url, title...
                if template_preprocess == 0:
                    if line != "--- template_end ---":
                        # detail = line.split("::")
                        
                
                        if len(line) > 1:   
                            append_next = 1
                            
                            if append_next == 1:
                                child = dict({})
                            
                            detail = line.split("::")      
                            
                            if(len(detail) > 1):
                                k = detail[0].strip()
                                v = detail[1].strip()   
                                
                                if v[0:8] == "math_viz":    
                                    i_file += 1
                                    math_module = v.split('\'')[1]
                                    child['file'] = f'{math_module}'
                                    child['title'] = k
                                    child['filePath'] = f'{url_prefix_math}/{math_module}'
                                    child['url'] = f'{url_prefix_math}/{math_module}'
                                    child['math_viz'] = True
                                    child['tutorial'] = f'{url_prefix_math}/{course_file_path}/file-{mod}-{i_file}.md'
                                
                                elif v[0:5] == "vimeo":
                                    i_file += 1
                                    vimeo_module = v.split('\'')[1]
                                    child['file'] = f'{vimeo_module}'
                                    child['title'] = k
                                    child['filePath'] = f'{vimeo_module}'
                                    child['url'] = f'{vimeo_module}'
                                    child['vimeo'] = True
                                    
                                elif v[0:2] == "ba": # assessment 
                                    i_file += 1
                                    # assessment_module = v.split('\'')[1]
                                    child['file'] = f'file-{mod}-{i_file}.ba'
                                    child['title'] = k
                                    child['filePath'] = f'{course_file_path}/file-{mod}-{i_file}.ba'
                                    child['url'] = f'{url_prefix}/{course_file_path}/file-{mod}-{i_file}.ba'
                                    child['assessment'] = f"load('./file-{mod}-{i_file}.json')"
                                
                                    
                                   # if len(assessment_module) > 0:
                                        
                                
                                elif v == "notebook":
                                    i_file += 1
                                    child['file'] = f'file-{mod}-{i_file}.ipynb'
                                    child['title'] = k
                                    child['filePath'] = f'{course_file_path}/file-{mod}-{i_file}.ipynb'
                                    child['url'] = f'{url_prefix}/{course_file_path}/file-{mod}-{i_file}.ipynb'
                                    child['code'] = True
                                    child['notebook'] = True
                                    child['tutorial'] = f'{url_prefix}/{course_file_path}/file-{mod}-{i_file}.md'
                                    
                                elif v == "code":
                                    i_file += 1
                                    child['file'] = f'file-{mod}-{i_file}.py'
                                    child['title'] = k
                                    child['filePath'] = f'{course_file_path}/file-{mod}-{i_file}.py'
                                    child['url'] = f'{url_prefix}/{course_file_path}/file-{mod}-{i_file}.py'
                                    child['code'] = True
                                    child['tutorial'] = f'{url_prefix}/{course_file_path}/file-{mod}-{i_file}.md'
                                
                                
                            else:
                                i_file += 1
                                child['file'] = f'file-{mod}-{i_file}.md'
                                child['title'] = line
                                child['filePath'] = f'{course_file_path}/file-{mod}-{i_file}.md'
                                child['url'] = f'{url_prefix}/{course_file_path}/file-{mod}-{i_file}.md'
                                child['tutorial'] = f'{url_prefix}/{course_file_path}/file-{mod}-{i_file}.md'
                                
                                

                            
                            # if(v == "true"): v = True
                            # if(v == "false"): v = False
                        
                            
                            # if k == "pull":
                            #     child.update(eval(v))
                            # else:
                            #     child[k] = v
                            
                            if append_next == 1:
                                append_next = 0
                                sectionObj['children'].append(child)
                                
    return sectionObj, sectionObj['fileName']


process_playlist_section('./Module-X.bpls')

({'fileName': 'Module-0',
  'section': 'Welcome',
  'children': [{'file': 'file-1-1.md',
    'title': 'Overview',
    'filePath': 'bignumber-material/SPR/mod1/file-1-1.md',
    'url': 'http://localhost:3001/bignumber-material/SPR/mod1/file-1-1.md',
    'tutorial': 'http://localhost:3001/bignumber-material/SPR/mod1/file-1-1.md'},
   {'file': 'file-1-2.py',
    'title': 'Overview Code',
    'filePath': 'bignumber-material/SPR/mod1/file-1-2.py',
    'url': 'http://localhost:3001/bignumber-material/SPR/mod1/file-1-2.py',
    'code': True,
    'tutorial': 'http://localhost:3001/bignumber-material/SPR/mod1/file-1-2.md'},
   {'file': 'file-1-3.ipynb',
    'title': 'Overview Notebook',
    'filePath': 'bignumber-material/SPR/mod1/file-1-3.ipynb',
    'url': 'http://localhost:3001/bignumber-material/SPR/mod1/file-1-3.ipynb',
    'code': True,
    'notebook': True,
    'tutorial': 'http://localhost:3001/bignumber-material/SPR/mod1/file-1-3.md'},
   {'file': 'file-1-4.ba',
    'title': 'Overview 

In [5]:
# Prepare new playlist dict
new_video_playlist = []

# Directory path
directory = './'

# Process all assessment files (.ba)
# File extension
extension = '.ba'

# Iterate over files in the directory
for filename in os.listdir(directory):
    if filename.endswith(extension):
        file_path = os.path.join(directory, filename)
        assessmentObj, assessmentFileName = process_assessment(file_path)
        save(assessmentObj, f'./{assessmentFileName}.json')


# Process all playlist section files (.ba)
# File extension
extension = '.bpls'

# Iterate over files in the directory
files = os.listdir(directory)
files = [filename for filename in files if filename.endswith(extension)]
files = sorted(files)

for filename in files:
    if filename.endswith(extension):
        file_path = os.path.join(directory, filename)
        sectionObj, sectionFileName = process_playlist_section(file_path)
       
        # Populate playlist dict
        new_video_playlist.append(sectionObj)
        
        save(sectionObj, f'./{sectionFileName}.json')

Saved file
Saved file
Saved file
Saved file
Saved file
Saved file


In [6]:
files

['Module-0.bpls',
 'Module-1.bpls',
 'Module-2.bpls',
 'Module-3.bpls',
 'Module-4.bpls']

In [7]:
enrollment_programs = load('../enrollment-programs.json')

In [8]:
enrollment_programs['programs']['SPR']['videoPlaylist']

[{'fileName': 'Module-0',
  'section': 'Welcome',
  'children': [{'file': 'Welcome',
    'preview': True,
    'url': 'https://player.vimeo.com/video/779911021?h=11699f372a',
    'title': 'Welcome Video',
    'duration': '02:54'}]},
 {'fileName': 'Module-1',
  'section': 'Module 1',
  'children': [{'file': 'file-1-1.md',
    'filePath': 'http://localhost:3001/bignumber-material/SPR/mod1/file-1-1.md',
    'url': 'http://localhost:3001/bignumber-material/SPR/mod1/file-1-1.md',
    'title': 'Overview',
    'tutorial': 'http://localhost:3001/bignumber-material/SPR/mod1/file-1-1.md',
    'duration': '0mins'},
   {'file': 'file-1-2.md',
    'filePath': 'http://localhost:3001/bignumber-material/SPR/mod1/file-1-2.md',
    'url': 'http://localhost:3001/bignumber-material/SPR/mod1/file-1-2.md',
    'title': 'Goal of Statistics (vid)',
    'tutorial': 'http://localhost:3001/bignumber-material/SPR/mod1/file-1-2.md',
    'duration': '0mins'},
   {'file': 'file-1-3.md',
    'filePath': 'http://localh

In [9]:
enrollment_programs['programs']['SPR']['videoPlaylist'] = new_video_playlist

In [10]:
enrollment_programs['programs']['SPR']['videoPlaylist']

[{'fileName': 'Module-0',
  'section': 'Welcome',
  'children': [{'file': 'Welcome',
    'preview': True,
    'url': 'https://player.vimeo.com/video/779911021?h=11699f372a',
    'title': 'Welcome Video',
    'duration': '02:54'}]},
 {'fileName': 'Module-1',
  'section': 'Module 1',
  'children': [{'file': 'M1c1.mov',
    'url': 'https://player.vimeo.com/video/797174163?h=c8f7d92f8d',
    'title': 'Inferring with Statistics',
    'duration': '01:59',
    'preview': True,
    'description': "In this video, we'll establish the definition of statistics."},
   {'file': 'M1c2.mov',
    'url': 'https://player.vimeo.com/video/797174300?h=62934b3418',
    'title': 'Statistics and Probability Theory',
    'duration': '03:09',
    'preview': True},
   {'file': 's1-6-1.mov',
    'url': 'https://player.vimeo.com/video/778897448?h=c5da8a109a',
    'title': 'Example: Calculating Probability',
    'duration': '02:51'},
   {'file': 'term1.mov',
    'url': 'https://player.vimeo.com/video/795400358?h=25e

In [11]:
save(enrollment_programs, '../enrollment-programs.json')

Saved file


In [12]:
json.dumps("true")

'"true"'